In [ ]:
# default_exp detect

In [ ]:
#hide
# """
# the weights and images are stored my Google Drive
# """
from google.colab import drive
drive.mount('/content/drive')

# hope you're already connected to the google drive!
%cd "/content/drive/MyDrive/Coding/ModelAssistedLabel/"

Mounted at /content/drive
/content/drive/MyDrive/Coding/ModelAssistedLabel


In [ ]:
#hide
# %matplotlib inline
%run "_Synch.ipynb"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Coding/ModelAssistedLabel
install nbdev: y
git commit -m: 3 runs available to compare
hooks
diffs
Converted 00_config.ipynb.
Converted 01_split.ipynb.
Converted 02_train.ipynb.
Converted 03_detect.ipynb.
Converted index.ipynb.
converting: /content/drive/My Drive/Coding/ModelAssistedLabel/03_detect.ipynb
converting /content/drive/My Drive/Coding/ModelAssistedLabel/index.ipynb to README.md


# Detecting Objects
> wrapping `yolov5/detect.py`

In [ ]:
#export

import os, torch
os.chdir("yolov5")
from pathlib import Path
from utils.plots import plot_one_box
from utils.general import check_img_size, non_max_suppression
from utils.datasets import LoadStreams, LoadImages
from utils.general import scale_coords, xyxy2xywh
from utils.torch_utils import select_device
from models.experimental import attempt_load
import cv2, os, base64, json
os.chdir("..")

os.system("pip install pillow")

class Detector:
  """A wrapper for training loading saved YOLOv5 weights
  
  requirements:
    GPU enabled"""
  def __init__(self, weight_path, conf_threshold = .4, iou_threshold = .45, imgsz = 416, save_dir="save_dir", write_annotated_images_to_disk=False):
    """ Constructor. I pulled the default numeric values above directly from the
    detect.py file. I added the option to save model output to both images and 
    to txt files

    Args:
      weight_path: the path to which the saved weights are stored
      conf_threshold: lower bound on the acceptable level of uncertainty for a 
                      bounding box
      iou_threshold: IoU helps determine how overlapped two shapes are.
      imgsz: resolution of image to process (assumes square)
      save_dir: where to write annotated images
      write_annotated_images_to_disk: save human-friendly annotated images to disk 
    """
    
    self.weight_path = weight_path
    self.conf_threshold = conf_threshold
    self.iou_threshold = iou_threshold
    self.imgsz = imgsz
    self.device = select_device()
    self.model = attempt_load(self.weight_path, map_location=self.device)  # load FP32 model
    self.names = self.model.module.names if hasattr(self.model, 'module') else self.model.names
    self.imgsz = check_img_size(self.imgsz, s=self.model.stride.max())  # check img_size
    self.half = self.device.type != 'cuda'
    if self.half:
      self.model.half()  # to FP16

    self.write_annotated_images_to_disk = write_annotated_images_to_disk
    self.save_dir = save_dir

  def make_dir(dir):
    """makes a directory provided that the directiory doesn't already exist
    
    Args:
      dir: Directory to create a path towards
    """
    if not os.path.exists(dir):
      os.makedirs(dir)

  def process_image(self, source):
    """Runs on the model with pre-specified weights an input image. See original
    detect.py for more details

    Args:
      source: A string path to pre-specified weights for the model
      save_unscuffed: create copy of the pre-image

    Reurns:
      A JSON-serializable object encoding bounding box information
    """
    override = None
    if os.path.exists(self.save_dir):
      override = input(f"Save directory '{self.save_dir}' exists. \n 'Enter' to continue, anything else to cancel operation")

    if override is None or len(override) == 0:
      return self.__process_image__(source)

    assert False, "this code shouldn't run"


  #not saving images speed up processing dramatically
  def __process_image__(self, source):
    "helper for process_image"
    results = []
    img = torch.zeros((1, 3, self.imgsz, self.imgsz), device=self.device)  # init img
    _ = self.model(img.half() if self.half else img) if self.device.type != 'cpu' else None  # run once
    dataset = LoadImages(source, img_size=self.imgsz)

    if self.write_annotated_images_to_disk:
      save_dir = Path(self.save_dir)
      Detector.make_dir(save_dir)

    for path, img, im0s, vid_cap in dataset:
      tmp = {}
      img = torch.from_numpy(img).to(self.device)
      img = img.half() if self.half else img.float()  # uint8 to fp16/32
      img /= 255.0  # 0 - 255 to 0.0 - 1.0
      
      if img.ndimension() == 3:
        img = img.unsqueeze(0)

      pred = self.model(img, augment=False)[0]
      pred = non_max_suppression(pred, self.conf_threshold, self.iou_threshold, agnostic=False)
      
      for i, det in enumerate(pred):
        p, s, im0, frame = path, '', im0s, getattr(dataset, 'frame', 0)
        p = Path(p) 
        
        s += '%gx%g ' % img.shape[2:]  # print string
        gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh

        if len(det):
          det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
          for c in det[:, -1].unique():
              n = (det[:, -1] == c).sum()  # detections per class
              s += f'{n} {self.names[int(c)]}s, '  # add to string
          
          if self.write_annotated_images_to_disk:
            tmp["unscuffed"] = f"{save_dir}/unscuffed-{p.name}"
            cv2.imwrite(tmp["unscuffed"], im0)

          tmp["predictions"] = []
          for *xyxy, conf, cls in reversed(det):
            xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
            line = (cls, *xywh, conf) # label format
            tmp["predictions"].append(('%g ' * len(line)).rstrip() % line)

            # save image with bboxes drawn on top
            if self.write_annotated_images_to_disk: 
              label = f'{self.names[int(cls)]} {conf:.2f}'
              plot_one_box(xyxy, im0, label=label, color=[0,0,200], line_thickness=5)
              tmp["labeled"] = f"{save_dir}/labeled-{p.name}"
              cv2.imwrite(tmp["labeled"], im0)

    return tmp

# Human-friendly Labels

Human-readable information about the class indentities is stored in the data.yaml folder. By default, the data.yaml file is created from the `Defaults` class. Let's take a look:

In [ ]:
from ModelAssistedLabel.core import Defaults
print(Defaults().data_yaml)

train: ../train/images
val: ../valid/images

nc: 10
names: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']


To convert the "names" variable to a python-friendly format, we do the following manipulation:

In [ ]:
import ast, re

# needs to be wrapped in quotes to parse as dict
substitute = "names"

#select last line
classlist = Defaults().data_yaml.split("\n")[-1]

#add quotes around `names` ONLY around the start of a string
classlist = re.sub('^%s' % substitute, f"'{substitute}'", classlist)

#surround the string in curly braces to tell python it's a dict
classlist = f"{{{classlist}}}"

# parse string as dict
classlist = ast.literal_eval(classlist)

And now here is the value of the classes as used by yolov5

In [ ]:
classlist

{'names': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']}

# Visualizing the YOLOv5 Output
 

In [ ]:
#export
from ModelAssistedLabel.detect import Detector
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import PIL

class Viewer:
  """ After supplying this class with the paths to (1) several pre-generated weights,
  and (2) several new images, you can create a model from each weight and a prediction
  for each image from each model.
  """
  def __init__(self, weight_path, class_arr):
    """
    constructor builds the detectors (may be relatively time-intensive) and stores 
    the targets

    Args:
      weights_paths: an array of paths to weights
    """
    self.detector = Detector(weight_path=weight_path)
    self.class_arr = class_arr
    self.last_result = []
  
  def process(self, images, plot_results=True, show_labels=True,figsize=(20,10)):
    """
    Runs an image set through this class's detector.

    Args:
      images: array of images
      plot_results: display the images with superimposed bounding boxes
      show_labels: whether or not to show the human-friendly labels about the bounding boxes
      figsize: passed to plt

    Returns:
      an array of dicts. Dicts relate each set of predictions to an image path
    """
    results = []
    for image in images:
      result = self.detector.process_image(image)
      results.append({"img path": image, "predictions": result})
      #re-calculate predictions with "de-normalized" values
      predictions = self.__yolov5_pred_to_standard__(image, result)["predictions"]
      #show the image with super-imposed bounding boxes
      if plot_results:
        Viewer.__plot_with_bbox__(image,predictions,show_labels=show_labels, figsize=figsize)
    self.last_result.append(results)

  def __plot_with_bbox__(img_path, predictions, show_labels, figsize):
    """display the rectangles on top of the image using pyplot

    Args:
      img_path: path to the image of interest
      predictions: the output from the Detector's process image
      figsize: parameter for fig
    """
    fig, ax = plt.subplots(figsize=figsize)
    #open/display the image
    im0 = PIL.Image.open(img_path)
    ax.imshow(im0)

    for prediction in predictions:
      # Create a Rectangle patch
      rect = patches.Rectangle((prediction['x'], prediction['y']), prediction['width'], prediction['height'], linewidth=1, edgecolor='r', facecolor='none')
      if show_labels:
        ax.annotate(prediction["class"], xy=(prediction['x'], prediction['y']-10), color='r', fontsize=20)
      # Add the patch to the Axes
      ax.add_patch(rect)

    plt.show()
    
  def __yolov5_pred_to_standard__(self, image, result):
    """
    The standard YOLOv5 coordinate format is normed to 1. Need to extract the 
    original's image width and height to convert to a standard cartesian plane.

    Args:
      image: path to image
      result: a dictionary that includes both 
          * a key called "filename" that points to the original image
          * a key called "predictions" created when the image is parsed with the
            YOLOv5 model 

    Returns:
      Convert the predictions converted to a full-scale Cartesian coordinate system.
    """
    out = {}
    out["image path"] = image
    PILim= PIL.Image.open(image)
    width, height = PILim.width, PILim.height

    out["predictions"] = []
    for prediction in result["predictions"]:
      bbox = prediction.split(" ")
      out["predictions"].append({
          "class": self.class_arr[int(bbox[0])],
          "confidence": float(bbox[5]),
          "height": int(PILim.height*float(bbox[4])),
          "width": int(PILim.width*float(bbox[3])),
          "x": int(PILim.width*(float(bbox[1]) - float(bbox[3])/2)),
          "y": int(PILim.height*(float(bbox[2]) - float(bbox[4])/2))
          })
    return out

Set up a `Viewer` object to investigate the behavior of a model

In [ ]:
weight_path = 'pre-trained weights/21-2-25 1k-digits YOLOv5-weights.pt'
v = Viewer([weight_path], classlist['names'])

Fusing layers... 


Grad a sample of some new images

In [ ]:
repos = []
repos.append("Image Repo/unlabeled/21-3-18 rowing 8-12 /")
repos.append("Image Repo/unlabeled/21-3-22 rowing (200) 1:53-7:00")
repos.append("Image Repo/unlabeled/21-3-22 rowing (200) 7:50-12:50")

Process the sample

In [ ]:
v.detector.process_image(samples[2])

image 1/1 /content/drive/My Drive/Coding/ModelAssistedLabel/Image Repo/unlabeled/21-3-22 rowing (200) 1:53-7:00/84.jpg: 

{'predictions': ['1 0.491797 0.257639 0.0195312 0.0736111 0.551745',
  '3 0.442969 0.250694 0.0203125 0.0763889 0.743457',
  '9 0.464453 0.252083 0.0210938 0.0791667 0.786505',
  '9 0.414844 0.247222 0.021875 0.0861111 0.791806']}

In [ ]:
import random
repo = repos[1]
new_images = [os.path.join(repo, x) for x in os.listdir(repo)]
samples = random.sample(new_images, 5)
v.process(samples, plot_results=True)

Output hidden; open in https://colab.research.google.com to view.

All the results are stored away in the `last_result` attribute

In [ ]:
v.last_result[-1]

[{'img path': 'Image Repo/unlabeled/21-3-18 rowing 8-12 /164.jpg',
  'predictions': {'predictions': ['2 0.263672 0.379861 0.0226563 0.0958333 0.546299',
    '2 0.323047 0.386806 0.0210938 0.0875 0.803381',
    '9 0.292969 0.381944 0.0234375 0.0888889 0.825718',
    '6 0.352344 0.386806 0.01875 0.0930556 0.8395']}},
 {'img path': 'Image Repo/unlabeled/21-3-18 rowing 8-12 /122.jpg',
  'predictions': {'predictions': ['2 0.304688 0.377778 0.021875 0.0833333 0.556505',
    '2 0.33125 0.382639 0.0203125 0.0847222 0.723794',
    '2 0.275781 0.373611 0.01875 0.0888889 0.842041',
    '1 0.357422 0.386806 0.0226563 0.0875 0.920106']}},
 {'img path': 'Image Repo/unlabeled/21-3-18 rowing 8-12 /16.jpg',
  'predictions': {'predictions': ['8 0.380859 0.388889 0.0226563 0.0888889 0.435701',
    '7 0.381641 0.389583 0.0226563 0.0902778 0.503729',
    '3 0.353906 0.381944 0.0203125 0.0833333 0.546678',
    '2 0.298047 0.377083 0.0210938 0.0930556 0.820826',
    '4 0.325 0.379861 0.021875 0.0875 0.867799